In [51]:
G = 'B' # Type of classical algebraic group, Input: string 'A','B','C', or 'D'
l = 3 # Rank, Input: positive integer

In [52]:
g = LieAlgebra(QQ, cartan_type=[G,l]) #, representation="matrix"

In [91]:
B3 = RootSystem(["B",3]).root_space()
roots=list(B3.positive_roots())
roots

[alpha[1],
 alpha[2],
 alpha[3],
 alpha[1] + alpha[2],
 alpha[2] + 2*alpha[3],
 alpha[2] + alpha[3],
 alpha[1] + alpha[2] + 2*alpha[3],
 alpha[1] + alpha[2] + alpha[3],
 alpha[1] + 2*alpha[2] + 2*alpha[3]]

In [53]:
g

Lie algebra of ['B', 3] in the Chevalley basis

In [19]:
g.lie_algebra_generators()

Finite family {alpha[1]: E[alpha[1]], alpha[2]: E[alpha[2]], alpha[3]: E[alpha[3]], -alpha[1]: E[-alpha[1]], -alpha[2]: E[-alpha[2]], -alpha[3]: E[-alpha[3]], alphacheck[1]: h1, alphacheck[2]: h2, alphacheck[3]: h3}

In [20]:
g.inject_variables()

Defining e1, e2, e3, f1, f2, f3, h1, h2, h3


In [67]:
n = LieAlgebra(QQ, cartan_type=['A',4], nilpotent=True)

In [ ]:
# Fix an enumeration of the positive roots s.t. ht(\beta_i)\leq ht(\beta_{i+1})

